In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from numpy.linalg import inv
from numpy.linalg import norm

def read_data(datafile):
    with open(datafile, 'r', encoding='utf-8-sig') as data_file:
        data = []
        for line in data_file:
            lines = line.split()
            lines = [float(i) for i in lines]
            data.append(lines)
        data = np.array(data)
    return data

def split_data(data):
    train = data[0:np.min(np.where(data[:,0]>0)),:]
    test = data[np.min(np.where(data[:,0]>0)):len(data),:]
    train_classes_t = int(np.max(train[:,len(train.T)-1]))
    test_classes_t = int(np.max(test[:,len(train.T)-1]))
    traindata_coly = list(train[:,len(train.T)-1].astype(int))
    testdata_coly = list(test[:,len(train.T)-1].astype(int))
    train_features = train[:,3:len(train.T)-1]
    test_features = test[:,3:len(train.T)-1]
    train_features = train_features.T
    test_features = test_features.T
    return train, test, train_classes_t, test_classes_t, traindata_coly, testdata_coly, train_features, test_features

def indices_to_one_hot(data, nb_classes):
    return np.eye(nb_classes+1)[np.array(data)]

def training(train_features, train_T):
    error_vec = []
    max_lambda = 0.5
    min_lambda = 0.1
    length = int(max_lambda/min_lambda)
    lambda_vec = np.linspace(min_lambda,max_lambda,length)
    a_vec = []

    for lambda1 in lambda_vec:
        sigma = 1
        K = np.zeros((len(train_features.T),len(train_features.T)))
        for i in np.arange(len(train_features.T)):
            for j in np.arange(len(train_features.T)):
                norma = norm(train_features[:,i]-train_features[:,j], ord=2)**2
                val = np.exp(-norma/(2*sigma**2))
                K[i,j] = val
        a = np.matmul(inv(K + lambda1*np.eye(len(K), len(K))), train_T.T)
        a_vec.append(a)
        y_pred = np.matmul(K, a)
        y_pred = y_pred.T
        e = np.eye(11,11)
        for i in np.arange(len(y_pred.T)):
                normi = []
                for k in np.arange(len(e)):
                    norma = norm(e[:,k]-y_pred[:,i], ord=2)**2
                    normi.append(norma)
                normi = np.array([normi])
                classi = np.argmin(normi)
                y_pred[:,i] = e[:,classi]
        wrongly_classified = train_T - y_pred
        count = 0
        for i in np.arange(len(train_features.T)):
            if sum(wrongly_classified[:,i] > 0) > 0:
                count = count + 1
        train_error = count/len(train_features.T)
        error_vec.append(train_error)
    lam = lambda_vec[np.argmin(error_vec)]
    a = a_vec[np.argmin(error_vec)]
    print("Training error =", error_vec[np.argmin(error_vec)], "Lambda =", lam)
    return y_pred.T, lam, e, sigma, a, K
    
def testing(lam, y_pred, train_features, test_features, e, sigma, a, K):
    k_test = np.zeros((len(test_features.T), len(train_features.T)))
    for i in np.arange(len(test_features.T)):
        for j in np.arange(len(train_features.T)):
            norma = norm(test_features[:,i]-train_features[:,j], ord=2)**2
            val = np.exp(-norma/(2*sigma**2))
            k_test[i,j] = val
    y_pred_test = np.matmul(k_test,a)
    y_pred_test = y_pred_test.T
    for p in np.arange(len(y_pred_test.T)):
        normi2 = []
        for m in np.arange(len(e)):
            norma2 = norm(e[:,m]-y_pred_test[:,p], ord=2)**2
            normi2.append(norma2)
        normi2 = np.array([normi2])
        classi2 = np.argmin(normi2)
        y_pred_test[:,p] = e[:,classi2]
    wrongly_classified2 = test_T - y_pred_test
    count = 0
    for i in np.arange(len(test_features.T)):
        if sum(wrongly_classified2[:,i] > 0) > 0:
            count = count + 1
    test_error = count/len(test_features.T)
    print("Test error =", test_error)
    
if __name__=="__main__":
    data = read_data('vowel-context.txt')
    train, test, train_classes_t, test_classes_t, traindata_coly, testdata_coly, train_features, test_features = split_data(data)
    train_T = indices_to_one_hot(traindata_coly, train_classes_t)
    test_T = indices_to_one_hot(testdata_coly, test_classes_t)
    train_T = train_T.T
    test_T = test_T.T
    y_pred, lam, e, sigma, a, K = training(train_features, train_T)
    testing(lam, y_pred, train_features, test_features, e, sigma, a, K)

Training error = 0.0 Lambda = 0.1
Test error = 0.37012987012987014
